In [1]:
! pip install fbprophet

In [2]:
from google.colab import drive
drive.mount('/gdrive', force_remount = True)

Mounted at /gdrive


In [3]:
import pandas as pd
from datetime import datetime
from fbprophet import Prophet

In [4]:
train = pd.read_csv('/gdrive/MyDrive/DACON 2021 농산물 가격예측 AI 경진대회/DATA/public_data/train.csv')
val = pd.read_csv('/gdrive/MyDrive/DACON 2021 농산물 가격예측 AI 경진대회/DATA/val.csv')
test = pd.read_csv('/gdrive/MyDrive/DACON 2021 농산물 가격예측 AI 경진대회/DATA/test.csv')

In [5]:
train.head(1)

,date,요일,배추_거래량(kg),배추_가격(원/kg),무_거래량(kg),무_가격(원/kg),양파_거래량(kg),양파_가격(원/kg),건고추_거래량(kg),건고추_가격(원/kg),마늘_거래량(kg),마늘_가격(원/kg),대파_거래량(kg),대파_가격(원/kg),얼갈이배추_거래량(kg),얼갈이배추_가격(원/kg),양배추_거래량(kg),양배추_가격(원/kg),깻잎_거래량(kg),깻잎_가격(원/kg),시금치_거래량(kg),시금치_가격(원/kg),미나리_거래량(kg),미나리_가격(원/kg),당근_거래량(kg),당근_가격(원/kg),파프리카_거래량(kg),파프리카_가격(원/kg),새송이_거래량(kg),새송이_가격(원/kg),팽이버섯_거래량(kg),팽이버섯_가격(원/kg),토마토_거래량(kg),토마토_가격(원/kg),청상추_거래량(kg),청상추_가격(원/kg),백다다기_거래량(kg),백다다기_가격(원/kg),애호박_거래량(kg),애호박_가격(원/kg),캠벨얼리_거래량(kg),캠벨얼리_가격(원/kg),샤인마스캇_거래량(kg),샤인마스캇_가격(원/kg)
0,2016-01-01,금요일,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
val.head(1)

,date,배추_거래량(kg),배추_가격(원/kg),무_거래량(kg),무_가격(원/kg),양파_거래량(kg),양파_가격(원/kg),건고추_거래량(kg),건고추_가격(원/kg),마늘_거래량(kg),마늘_가격(원/kg),대파_거래량(kg),대파_가격(원/kg),얼갈이배추_거래량(kg),얼갈이배추_가격(원/kg),양배추_거래량(kg),양배추_가격(원/kg),깻잎_거래량(kg),깻잎_가격(원/kg),시금치_거래량(kg),시금치_가격(원/kg),미나리_거래량(kg),미나리_가격(원/kg),당근_거래량(kg),당근_가격(원/kg),파프리카_거래량(kg),파프리카_가격(원/kg),새송이_거래량(kg),새송이_가격(원/kg),팽이버섯_거래량(kg),팽이버섯_가격(원/kg),토마토_거래량(kg),토마토_가격(원/kg),청상추_거래량(kg),청상추_가격(원/kg),백다다기_거래량(kg),백다다기_가격(원/kg),애호박_거래량(kg),애호박_가격(원/kg),캠벨얼리_거래량(kg),캠벨얼리_가격(원/kg),샤인마스캇_거래량(kg),샤인마스캇_가격(원/kg)
0,20200929,2064183.3,1786.71616,2424383.2,1190.203517,1209647.0,965.516611,2842.8,25872.801463,160209.8,5403.793994,913019.3,2682.218223,453688.3,2459.779675,723889.2,1584.470498,128308.2,11298.098259,318908.95,6176.525453,98351.0,8190.398044,382908.0,1742.80359,178569.6,7579.169517,295588.5,3343.379225,164620.4,3102.358954,205583.0,4268.396657,76702.8,3678,375798.6,2794,450576.5,2350,495845.2,3937,259765.0,11400


In [7]:
test.head(1)

,date,배추_거래량(kg),배추_가격(원/kg),무_거래량(kg),무_가격(원/kg),양파_거래량(kg),양파_가격(원/kg),건고추_거래량(kg),건고추_가격(원/kg),마늘_거래량(kg),마늘_가격(원/kg),대파_거래량(kg),대파_가격(원/kg),얼갈이배추_거래량(kg),얼갈이배추_가격(원/kg),양배추_거래량(kg),양배추_가격(원/kg),깻잎_거래량(kg),깻잎_가격(원/kg),시금치_거래량(kg),시금치_가격(원/kg),미나리_거래량(kg),미나리_가격(원/kg),당근_거래량(kg),당근_가격(원/kg),파프리카_거래량(kg),파프리카_가격(원/kg),새송이_거래량(kg),새송이_가격(원/kg),팽이버섯_거래량(kg),팽이버섯_가격(원/kg),토마토_거래량(kg),토마토_가격(원/kg),청상추_거래량(kg),청상추_가격(원/kg),백다다기_거래량(kg),백다다기_가격(원/kg),애호박_거래량(kg),애호박_가격(원/kg),캠벨얼리_거래량(kg),캠벨얼리_가격(원/kg),샤인마스캇_거래량(kg),샤인마스캇_가격(원/kg)
0,20210101,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0


In [8]:
train = train.drop(['요일'], axis='columns')
train.columns

Index(['date', '배추_거래량(kg)', '배추_가격(원/kg)', '무_거래량(kg)', '무_가격(원/kg)',
       '양파_거래량(kg)', '양파_가격(원/kg)', '건고추_거래량(kg)', '건고추_가격(원/kg)',
       '마늘_거래량(kg)', '마늘_가격(원/kg)', '대파_거래량(kg)', '대파_가격(원/kg)',
       '얼갈이배추_거래량(kg)', '얼갈이배추_가격(원/kg)', '양배추_거래량(kg)', '양배추_가격(원/kg)',
       '깻잎_거래량(kg)', '깻잎_가격(원/kg)', '시금치_거래량(kg)', '시금치_가격(원/kg)',
       '미나리_거래량(kg)', '미나리_가격(원/kg)', '당근_거래량(kg)', '당근_가격(원/kg)',
       '파프리카_거래량(kg)', '파프리카_가격(원/kg)', '새송이_거래량(kg)', '새송이_가격(원/kg)',
       '팽이버섯_거래량(kg)', '팽이버섯_가격(원/kg)', '토마토_거래량(kg)', '토마토_가격(원/kg)',
       '청상추_거래량(kg)', '청상추_가격(원/kg)', '백다다기_거래량(kg)', '백다다기_가격(원/kg)',
       '애호박_거래량(kg)', '애호박_가격(원/kg)', '캠벨얼리_거래량(kg)', '캠벨얼리_가격(원/kg)',
       '샤인마스캇_거래량(kg)', '샤인마스캇_가격(원/kg)'],
      dtype='object')

In [9]:
len(train['date'])+len(val['date'])+len(test['date'])

2135

In [10]:
train['date'] = train['date'].astype('str')
train['date'] = pd.to_datetime(train['date'])
train.head()

,date,배추_거래량(kg),배추_가격(원/kg),무_거래량(kg),무_가격(원/kg),양파_거래량(kg),양파_가격(원/kg),건고추_거래량(kg),건고추_가격(원/kg),마늘_거래량(kg),마늘_가격(원/kg),대파_거래량(kg),대파_가격(원/kg),얼갈이배추_거래량(kg),얼갈이배추_가격(원/kg),양배추_거래량(kg),양배추_가격(원/kg),깻잎_거래량(kg),깻잎_가격(원/kg),시금치_거래량(kg),시금치_가격(원/kg),미나리_거래량(kg),미나리_가격(원/kg),당근_거래량(kg),당근_가격(원/kg),파프리카_거래량(kg),파프리카_가격(원/kg),새송이_거래량(kg),새송이_가격(원/kg),팽이버섯_거래량(kg),팽이버섯_가격(원/kg),토마토_거래량(kg),토마토_가격(원/kg),청상추_거래량(kg),청상추_가격(원/kg),백다다기_거래량(kg),백다다기_가격(원/kg),애호박_거래량(kg),애호박_가격(원/kg),캠벨얼리_거래량(kg),캠벨얼리_가격(원/kg),샤인마스캇_거래량(kg),샤인마스캇_가격(원/kg)
0,2016-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2016-01-02,80860.0,329.0,80272.0,360.0,122787.5,1281.0,3.0,11000.0,15019.0,5475.0,92334.0,1704.0,6359.0,1331.0,40028.0,348.0,4374.9,13242.0,16550.5,2339.0,10528.0,1729.0,13885.0,804.0,3853.0,3703.0,15797.0,2576.0,14634.0,1474.0,30950.0,1621.0,5125.0,9235.0,434.0,2109.0,19159.0,2414.0,880.0,2014.0,0.0,0.0
2,2016-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2016-01-04,1422742.5,478.0,1699653.7,382.0,2315079.0,1235.0,699.0,4464.0,141638.0,5210.0,994328.1,1716.0,262615.5,1212.0,1074699.1,345.0,122613.5,9923.0,427435.1,2153.0,82113.5,3960.0,558950.2,794.0,104930.3,4871.0,277326.5,2440.0,159800.0,1750.0,291057.0,1834.0,38525.5,7631.0,500702.0,2046.0,620539.0,2018.0,2703.8,3885.0,0.0,0.0
4,2016-01-05,1167241.0,442.0,1423482.3,422.0,2092960.1,1213.0,1112.6,4342.0,126207.8,5387.0,787716.0,1715.0,221850.5,1197.0,825681.9,350.0,79055.9,9529.0,334636.8,2220.0,80144.0,3333.0,444353.7,763.0,100699.5,5129.0,218465.2,2437.0,153084.0,1822.0,194626.5,1833.0,32615.0,6926.0,147638.0,2268.0,231958.0,2178.0,8810.0,2853.0,0.0,0.0


In [11]:
val['date'] = val['date'].astype('str')
val['date'] = pd.to_datetime(val['date'])
val.head(2)

,date,배추_거래량(kg),배추_가격(원/kg),무_거래량(kg),무_가격(원/kg),양파_거래량(kg),양파_가격(원/kg),건고추_거래량(kg),건고추_가격(원/kg),마늘_거래량(kg),마늘_가격(원/kg),대파_거래량(kg),대파_가격(원/kg),얼갈이배추_거래량(kg),얼갈이배추_가격(원/kg),양배추_거래량(kg),양배추_가격(원/kg),깻잎_거래량(kg),깻잎_가격(원/kg),시금치_거래량(kg),시금치_가격(원/kg),미나리_거래량(kg),미나리_가격(원/kg),당근_거래량(kg),당근_가격(원/kg),파프리카_거래량(kg),파프리카_가격(원/kg),새송이_거래량(kg),새송이_가격(원/kg),팽이버섯_거래량(kg),팽이버섯_가격(원/kg),토마토_거래량(kg),토마토_가격(원/kg),청상추_거래량(kg),청상추_가격(원/kg),백다다기_거래량(kg),백다다기_가격(원/kg),애호박_거래량(kg),애호박_가격(원/kg),캠벨얼리_거래량(kg),캠벨얼리_가격(원/kg),샤인마스캇_거래량(kg),샤인마스캇_가격(원/kg)
0,2020-09-29,2064183.3,1786.716160,2424383.2,1190.203517,1209647.0,965.516611,2842.8,25872.801463,160209.8,5403.793994,913019.3,2682.218223,453688.3,2459.779675,723889.2,1584.470498,128308.2,11298.098259,318908.95,6176.525453,98351.0,8190.398044,382908.0,1742.803590,178569.6,7579.169517,295588.5,3343.379225,164620.4,3102.358954,205583.0,4268.396657,76702.8,3678,375798.6,2794,450576.5,2350,495845.2,3937,259765.0,11400
1,2020-09-30,259065.3,1550.579333,1044460.0,984.487237,136696.4,856.922026,35.0,9928.571429,48296.7,5575.309286,497001.5,2774.352659,218194.1,1512.730784,244625.0,1378.640589,96794.4,6476.310727,170331.10,3312.704356,61505.0,2037.861979,71892.6,956.189065,101203.4,4769.508485,155808.6,2561.098354,82371.8,2400.171940,140376.0,3550.460691,53183.2,1892,387513.8,1858,386869.7,1311,263761.0,3885,121859.6,11649


In [12]:
test['date'] = test['date'].astype('str')
test['date'] = pd.to_datetime(test['date'])
test.head(2)

,date,배추_거래량(kg),배추_가격(원/kg),무_거래량(kg),무_가격(원/kg),양파_거래량(kg),양파_가격(원/kg),건고추_거래량(kg),건고추_가격(원/kg),마늘_거래량(kg),마늘_가격(원/kg),대파_거래량(kg),대파_가격(원/kg),얼갈이배추_거래량(kg),얼갈이배추_가격(원/kg),양배추_거래량(kg),양배추_가격(원/kg),깻잎_거래량(kg),깻잎_가격(원/kg),시금치_거래량(kg),시금치_가격(원/kg),미나리_거래량(kg),미나리_가격(원/kg),당근_거래량(kg),당근_가격(원/kg),파프리카_거래량(kg),파프리카_가격(원/kg),새송이_거래량(kg),새송이_가격(원/kg),팽이버섯_거래량(kg),팽이버섯_가격(원/kg),토마토_거래량(kg),토마토_가격(원/kg),청상추_거래량(kg),청상추_가격(원/kg),백다다기_거래량(kg),백다다기_가격(원/kg),애호박_거래량(kg),애호박_가격(원/kg),캠벨얼리_거래량(kg),캠벨얼리_가격(원/kg),샤인마스캇_거래량(kg),샤인마스캇_가격(원/kg)
0,2021-01-01,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
1,2021-01-02,24401.5,388.534188,21461.0,559.526583,20606.0,1058.393186,231.0,23147.186147,2408.0,5129.931063,27859.0,2384.020245,746.0,1005.969169,23734.0,744.130362,4079.5,13150.276995,9920.0,2642.957863,7033.0,1614.409498,2785.0,822.829084,7849.0,4884.721875,12431.0,2365.326281,10077.0,2243.748834,25290.0,1914.895216,8447.0,4652,1490.0,2491,20982.0,2686,0.0,0,1820.0,11574


In [13]:
data = pd.concat([train, val, test], ignore_index=True)
data.head(2)

,date,배추_거래량(kg),배추_가격(원/kg),무_거래량(kg),무_가격(원/kg),양파_거래량(kg),양파_가격(원/kg),건고추_거래량(kg),건고추_가격(원/kg),마늘_거래량(kg),마늘_가격(원/kg),대파_거래량(kg),대파_가격(원/kg),얼갈이배추_거래량(kg),얼갈이배추_가격(원/kg),양배추_거래량(kg),양배추_가격(원/kg),깻잎_거래량(kg),깻잎_가격(원/kg),시금치_거래량(kg),시금치_가격(원/kg),미나리_거래량(kg),미나리_가격(원/kg),당근_거래량(kg),당근_가격(원/kg),파프리카_거래량(kg),파프리카_가격(원/kg),새송이_거래량(kg),새송이_가격(원/kg),팽이버섯_거래량(kg),팽이버섯_가격(원/kg),토마토_거래량(kg),토마토_가격(원/kg),청상추_거래량(kg),청상추_가격(원/kg),백다다기_거래량(kg),백다다기_가격(원/kg),애호박_거래량(kg),애호박_가격(원/kg),캠벨얼리_거래량(kg),캠벨얼리_가격(원/kg),샤인마스캇_거래량(kg),샤인마스캇_가격(원/kg)
0,2016-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2016-01-02,80860.0,329.0,80272.0,360.0,122787.5,1281.0,3.0,11000.0,15019.0,5475.0,92334.0,1704.0,6359.0,1331.0,40028.0,348.0,4374.9,13242.0,16550.5,2339.0,10528.0,1729.0,13885.0,804.0,3853.0,3703.0,15797.0,2576.0,14634.0,1474.0,30950.0,1621.0,5125.0,9235.0,434.0,2109.0,19159.0,2414.0,880.0,2014.0,0.0,0.0


In [14]:
data.tail(4)

,date,배추_거래량(kg),배추_가격(원/kg),무_거래량(kg),무_가격(원/kg),양파_거래량(kg),양파_가격(원/kg),건고추_거래량(kg),건고추_가격(원/kg),마늘_거래량(kg),마늘_가격(원/kg),대파_거래량(kg),대파_가격(원/kg),얼갈이배추_거래량(kg),얼갈이배추_가격(원/kg),양배추_거래량(kg),양배추_가격(원/kg),깻잎_거래량(kg),깻잎_가격(원/kg),시금치_거래량(kg),시금치_가격(원/kg),미나리_거래량(kg),미나리_가격(원/kg),당근_거래량(kg),당근_가격(원/kg),파프리카_거래량(kg),파프리카_가격(원/kg),새송이_거래량(kg),새송이_가격(원/kg),팽이버섯_거래량(kg),팽이버섯_가격(원/kg),토마토_거래량(kg),토마토_가격(원/kg),청상추_거래량(kg),청상추_가격(원/kg),백다다기_거래량(kg),백다다기_가격(원/kg),애호박_거래량(kg),애호박_가격(원/kg),캠벨얼리_거래량(kg),캠벨얼리_가격(원/kg),샤인마스캇_거래량(kg),샤인마스캇_가격(원/kg)
2131,2021-11-01,1754622.65,612.637059,2552118.1,440.219583,2078178.75,858.803649,1708.7,10578.912624,102888.8,6413.702823,698053.3,1312.295918,316754.5,1362.837096,560305.2,489.903553,103828.5,8443.715685,183042.20,3692.503931,67042.2,5010.249619,518826.0,854.891862,193746.9,3133.891907,181826.5,2991.918087,116035.5,2852.920942,250732.0,3212.498405,49776.0,5718.0,400605.4,2154.0,445453.8,2601.0,11186.0,6119.0,118166.9,10524.0
2132,2021-11-02,1770020.10,608.982626,2418927.8,400.107347,1705887.20,867.063870,1510.2,11978.239968,157062.8,6286.677571,631986.7,1283.237336,274672.5,1255.009864,659218.9,389.801322,70855.2,7279.044149,178767.19,3363.987452,69876.7,3770.678610,453537.0,794.008804,135876.2,2765.279799,166897.8,2961.459270,113226.5,2756.150565,235696.9,2963.519631,41930.0,4542.0,242966.0,2101.0,277731.8,2262.0,8196.0,6383.0,105298.1,10522.0
2133,2021-11-03,1771095.50,657.370413,2297083.1,420.386008,1600168.70,886.031877,2416.1,9445.205496,107988.1,6856.532924,644889.4,1301.294473,271088.0,1222.720065,706836.0,372.793525,72748.6,6800.928664,184850.33,3167.000762,68183.8,4726.919444,330936.0,842.595562,193116.8,2762.874892,149810.4,2957.834883,118209.0,2669.843717,200195.0,2905.264267,46066.5,3826.0,231324.0,2390.0,272012.4,2321.0,8646.0,6524.0,97839.1,11222.0
2134,2021-11-04,1807489.00,684.641065,2192418.2,437.379582,1649971.90,851.705153,1741.3,11155.323609,120153.6,6287.203005,615145.7,1287.175615,262560.0,1189.192013,607451.9,388.759792,66805.2,6628.768584,181365.84,3030.049203,38484.6,5262.935122,262702.0,795.081766,153850.3,2405.021193,136146.0,2929.453623,105283.0,2573.589829,152301.0,2981.572019,40895.8,3385.0,206003.0,2904.0,205910.6,2569.0,6933.0,6529.0,88592.5,10837.0


In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2135 entries, 0 to 2134
Data columns (total 43 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   date            2135 non-null   datetime64[ns]
 1   배추_거래량(kg)      2135 non-null   float64       
 2   배추_가격(원/kg)     2135 non-null   float64       
 3   무_거래량(kg)       2135 non-null   float64       
 4   무_가격(원/kg)      2135 non-null   float64       
 5   양파_거래량(kg)      2135 non-null   float64       
 6   양파_가격(원/kg)     2135 non-null   float64       
 7   건고추_거래량(kg)     2135 non-null   float64       
 8   건고추_가격(원/kg)    2135 non-null   float64       
 9   마늘_거래량(kg)      2135 non-null   float64       
 10  마늘_가격(원/kg)     2135 non-null   float64       
 11  대파_거래량(kg)      2135 non-null   float64       
 12  대파_가격(원/kg)     2135 non-null   float64       
 13  얼갈이배추_거래량(kg)   2135 non-null   float64       
 14  얼갈이배추_가격(원/kg)  2135 non-null   float64       
 15  양배추_

In [16]:
final = pd.read_csv('/gdrive/MyDrive/DACON 2021 농산물 가격예측 AI 경진대회/DATA/sample_submission.csv')
final.columns

Index(['예측대상일자', '배추_가격(원/kg)', '무_가격(원/kg)', '양파_가격(원/kg)', '건고추_가격(원/kg)',
       '마늘_가격(원/kg)', '대파_가격(원/kg)', '얼갈이배추_가격(원/kg)', '양배추_가격(원/kg)',
       '깻잎_가격(원/kg)', '시금치_가격(원/kg)', '미나리_가격(원/kg)', '당근_가격(원/kg)',
       '파프리카_가격(원/kg)', '새송이_가격(원/kg)', '팽이버섯_가격(원/kg)', '토마토_가격(원/kg)',
       '청상추_가격(원/kg)', '백다다기_가격(원/kg)', '애호박_가격(원/kg)', '캠벨얼리_가격(원/kg)',
       '샤인마스캇_가격(원/kg)'],
      dtype='object')

In [17]:
from datetime import timedelta
from datetime import datetime
for i in range(1,22):
  df = pd.DataFrame(columns=['ds', 'y'])
  df['ds'] = data['date']
  df['y'] = data[data.columns[i*2]]
  model = Prophet(daily_seasonality=True)
  model.fit(df)
  future = model.make_future_dataframe(periods=28)
  forecast = model.predict(future)
  result = forecast['yhat'].tail(28)
  final.loc[final['예측대상일자']==(datetime.today() - timedelta(1)).strftime('%Y-%m-%d')+'+1week',final.columns[i]]=result.iloc[6]
  final.loc[final['예측대상일자']==(datetime.today() - timedelta(1)).strftime('%Y-%m-%d')+'+2week',final.columns[i]]=result.iloc[13]
  final.loc[final['예측대상일자']==(datetime.today() - timedelta(1)).strftime('%Y-%m-%d')+'+4week',final.columns[i]]=result.iloc[27]
final.tail(12)

INFO:numexpr.utils:NumExpr defaulting to 2 threads.


,예측대상일자,배추_가격(원/kg),무_가격(원/kg),양파_가격(원/kg),건고추_가격(원/kg),마늘_가격(원/kg),대파_가격(원/kg),얼갈이배추_가격(원/kg),양배추_가격(원/kg),깻잎_가격(원/kg),시금치_가격(원/kg),미나리_가격(원/kg),당근_가격(원/kg),파프리카_가격(원/kg),새송이_가격(원/kg),팽이버섯_가격(원/kg),토마토_가격(원/kg),청상추_가격(원/kg),백다다기_가격(원/kg),애호박_가격(원/kg),캠벨얼리_가격(원/kg),샤인마스캇_가격(원/kg)
216,2021-11-01+1week,341.542661,326.625305,922.773487,15336.160004,6226.736950,1904.351440,1083.158917,511.598263,6071.642868,3356.599659,4609.045455,883.194654,3128.950502,2740.755113,2146.466983,2048.361237,3801.800822,1727.170001,1792.452135,4459.294699,13005.315824
217,2021-11-01+2week,341.093454,316.454854,923.935540,15158.235936,6224.359683,1935.433466,1052.541941,517.397837,5533.295370,3159.233272,5119.852069,817.137746,2518.432500,2639.187263,1991.697496,1968.952283,3485.000066,1850.898899,1657.045177,4392.063151,13362.316101
218,2021-11-01+4week,329.834216,311.345313,942.896334,14433.304516,6207.678048,1837.699110,1094.534829,450.306888,5676.339234,3188.119957,5453.839867,760.831583,2568.155237,2535.490707,1810.387363,2019.378629,3887.142201,2163.953322,1864.289156,4618.637446,13604.420401
219,2021-11-02+1week,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
220,2021-11-02+2week,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
221,2021-11-02+4week,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
222,2021-11-03+1week,343.486938,323.760866,933.521987,14412.399832,6295.041764,1766.061264,994.768689,488.087746,5872.233167,3092.943318,4457.795163,797.002662,2895.779026,2721.111564,2057.057153,2014.751303,3335.166508,1777.316696,1806.789288,4427.277057,12596.043248
223,2021-11-03+2week,337.368636,312.507317,936.622380,14266.980375,6273.844481,1778.187090,976.645560,488.871180,5418.784205,2928.288315,4953.067223,737.360178,2387.705853,2615.372585,1899.175402,1957.685359,3209.590240,1928.226464,1720.753764,4380.822970,12892.947418
224,2021-11-03+4week,325.279040,315.260527,956.377498,13507.482887,6274.146965,1667.775537,1023.286446,413.136926,5809.978904,3019.300489,5081.924981,693.947821,2646.549114,2553.962126,1759.794544,2016.376846,3600.520530,2195.342739,1977.283268,4604.120017,13046.221889
225,2021-11-04+1week,338.237529,333.765549,912.108082,14958.420361,6155.889920,1846.687717,980.089949,481.441521,5758.106347,3060.821431,4579.695530,803.318539,2761.977479,2691.331040,1999.240223,2063.233855,3310.357690,1849.216217,1831.910648,4315.268562,12754.083947


In [18]:
final.to_csv('/gdrive/MyDrive/DACON 2021 농산물 가격예측 AI 경진대회/DATA/sample_submission.csv', index=None)